In [2]:
!pip install -q skallel

In [3]:
from dask_kubernetes import KubeCluster
from dask.distributed import Client, performance_report
cluster = KubeCluster(n_workers=20,
                      env={'EXTRA_PIP_PACKAGES': 'skallel'})
client = Client(cluster)
# client = Client()
client

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at: tcp://10.33.122.115:46447
distributed.scheduler - INFO -   dashboard at:                     :8787
distributed.scheduler - INFO - Receive client connection: Client-842de894-2549-11eb-82c8-debd36506f5e
distributed.core - INFO - Starting established connection


Client Scheduler: tcp://10.33.122.115:46447 Dashboard: /user/alimanfoo@googlemail.com/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


distributed.scheduler - INFO - Register tcp://10.33.123.118:46293
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.33.123.118:46293
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.33.186.4:45603
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.33.186.4:45603
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.33.176.4:45019
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.33.176.4:45019
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.33.121.99:46231
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.33.121.99:46231
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.33.164.4:41737
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.33.164.4:41737
distrib

In [7]:
import zarr
import fsspec
import dask.array as da
import skallel

/opt/conda/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
store = fsspec.get_mapper('gs://ag1000g-release/phase2.AR1/variation/main/zarr/all/ag1000g.phase2.ar1')
callset_snps = zarr.open_consolidated(store=store)
gt = da.from_array(callset_snps['2R/calldata/GT'])
gt

,Array,Chunk
Bytes,56.57 GB,63.96 MB
Shape,"(24767689, 1142, 2)","(524288, 61, 2)"
Count,913 Tasks,912 Chunks
Type,int8,numpy.ndarray


In [43]:
gt = gt.rechunk((524288 // 8, -1, -1))
gt

,Array,Chunk
Bytes,56.57 GB,149.68 MB
Shape,"(24767689, 1142, 2)","(65536, 1142, 2)"
Count,34807 Tasks,378 Chunks
Type,int8,numpy.ndarray


In [44]:
# # convert to alt counts
# gn = (gt > 0).sum(axis=2)
# gn

In [48]:
dist = skallel.pairwise_distance(
    gt[:3_000_000, :, 1], 
    metric='euclidean')
dist

dask.array<sqrt, shape=(651511,), dtype=float64, chunksize=(651511,), chunktype=numpy.ndarray>

In [49]:
import dask
dask.config.set({"optimization.fuse.ave-width": 5})

In [50]:
%%time
dist.compute()

distributed.scheduler - INFO - Remove worker tcp://10.33.214.2:35205
distributed.core - INFO - Removing comms to tcp://10.33.214.2:35205


KeyboardInterrupt: 

In [51]:
cluster.adapt()

distributed.scheduler - INFO - Retire worker names (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19)
distributed.scheduler - INFO - Retire workers {<Worker 'tcp://10.33.120.136:33207', memory: 0, processing: 0>, <Worker 'tcp://10.33.217.2:35181', memory: 0, processing: 0>, <Worker 'tcp://10.33.121.99:46231', memory: 0, processing: 0>, <Worker 'tcp://10.33.213.2:37745', memory: 0, processing: 0>, <Worker 'tcp://10.33.164.4:42029', memory: 0, processing: 0>, <Worker 'tcp://10.33.123.118:46293', memory: 0, processing: 0>, <Worker 'tcp://10.33.218.2:34487', memory: 0, processing: 0>, <Worker 'tcp://10.33.195.4:43463', memory: 0, processing: 0>, <Worker 'tcp://10.33.219.2:42119', memory: 0, processing: 0>, <Worker 'tcp://10.33.220.2:34697', memory: 0, processing: 0>, <Worker 'tcp://10.33.186.4:37955', memory: 0, processing: 0>, <Worker 'tcp://10.33.167.4:36365', memory: 0, processing: 0>, <Worker 'tcp://10.33.216.2:33211', memory: 0, processing: 0>, <Worker 'tcp://10.33